# Quantization Stable Diffusion

_Authored by: [Thomas Liang](https://github.com/thliang01)_


- [ ] TODO: write description and quantization stable diffusion models

## Install required python package

In [1]:
! pip install --upgrade diffusers accelerate transformers safetensors
! pip install -q numpy Pillow

## Import modules

In [2]:
import torch
import numpy as np
import os

import time

from PIL import Image
from IPython import display as IPdisplay
from tqdm.auto import tqdm

from diffusers import DiffusionPipeline
from transformers import logging

logging.set_verbosity_error()

### Check CUDA is available

In [3]:
print(torch.cuda.is_available())

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

True


## Base Model

In [5]:
model_name_or_path = "stabilityai/stable-diffusion-xl-base-1.0"

pipe = DiffusionPipeline.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16, 
    variant="fp16",
    use_safetensors=True, 
).to(device)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

## Display_images

In [7]:
prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]
image

  0%|          | 0/50 [00:00<?, ?it/s]